In [1]:
%load_ext autoreload
%autoreload 2
from SQLParser import SQLParser, WithStatementStyle, HandlerSQL
import re

In [2]:
sql = WithStatementStyle()

In [3]:
raw1 = '''
WITH ORDERED_MOVINGS AS (
	SELECT
		t.subs_id
		, t.start_dttm
		, t.start_date
		, t.end_dttm
		, t.a_number
		, t.b_number
	FROM PRD2_DDS_V.USAGE AS t
	WHERE t.start_date BETWEEN DATE'2022-10-01' AND DATE'2022-10-31'
		AND t.traffic_type_id = 2
		AND (t.b_number = '7757' OR t.b_number = 11)
		AND (t.a_number IN ('312', '7757') OR Length(t.a_number) = 11)
        AND (t.b_number1 = '7757' OR t.b_number1 = 11)
        )

select * from ORDERED_MOVINGS
'''

In [4]:
d = sql.default_string(raw1)

In [7]:
def where_and_or_rule(raw):
    assert isinstance(raw, str), 'raw must be type of string'
    
    script_by_rows = [tup for tup in raw.split('\n') if len(tup) > 0]
    where_keywords = ['WHERE ', 'WHERE '] # в tuple нужно передать больше одного значения
    where_end_group_by = ['GROUP BY ', 'ORDER BY ']    
    
    where_groups = []
    counter = 0
    for ind in range(0, len(script_by_rows)-1):
        
        # если строка начинается с where, то запоминаем ее индекс
        if script_by_rows[ind].startswith(tuple(where_keywords)):
            counter = ind
            continue
        
        # если впереди group by, то записываем группу и обнуляем счетчик
        if script_by_rows[ind+1].startswith(tuple(where_end_group_by)) :
            where_groups.append(script_by_rows[counter:ind+1])
            counter = 0
            continue
        
        # если кол-во строк равно индексу (с поправкой) и счетчик не равен 0 (значит запрос заканчивается на выражении where), то записываем группу
        if len(script_by_rows)-2 == ind and counter != 0:
            where_groups.append(script_by_rows[counter:])
            counter = 0
    
    counter = 0
    and_groups = []
    for group in where_groups:
        for ind in range(0, len(group)-1):
            if 'AND (' in group[ind] and counter == 0:
                counter = ind
            if 'AND (' in group[ind+1] and counter != 0:
                and_groups.append(group[counter:ind+1])
                counter = 0
            if len(group)-2 == ind and counter != 0:
                and_groups.append(group[counter:])
    
    for group in and_groups:
        for ind in range(0, len(group)):
            if 'AND (' in group[ind]:
                re_val = group[ind].replace('(', '\(').replace(')', '\)')
                pattern = re.compile(f'{re_val}')
                spaces = 8 * ' '
                right_val = group[ind].strip().replace('(', '(\n' + spaces)
                raw = pattern.sub(right_val, raw)
            if len(group) - 1 > ind > 0:
                re_val = group[ind].replace('(', '\(').replace(')', '\)')
                pattern = re.compile(f'{re_val}')
                spaces = 8 * ' '
                right_val = spaces + group[ind].strip()
                raw = pattern.sub(right_val, raw, 1)
            if len(group) - 1 == ind:
                #print(group[ind], 'last')
                re_val = group[ind].replace('(', '\(').replace(')', '\)')
                pattern = re.compile(f'{re_val}')
                spaces = 4 * ' '
                start_spaces = 8 * ' '
                right_val = start_spaces + group[ind][:-1].strip() + '\n' + spaces + ')'
                raw = pattern.sub(right_val, raw, 1)
            
    return raw

In [8]:
print(where_and_or_rule(d))


SELECT t.subs_id ,
       t.start_dttm ,
       t.start_date ,
       t.end_dttm ,
       t.a_number ,
       t.b_number
FROM prd2_dds_v.usage AS t
WHERE t.start_date BETWEEN DATE'2022-10-01' AND DATE'2022-10-31'
    AND t.traffic_type_id = 2
AND (
        t.b_number = '7757'
        OR t.b_numbe = 11
    )
AND (
        t.a_number IN (
        '312',
        '7757')
        OR length(t.a_number) = 11
    )


In [79]:
sql.run_code_style_for_with(t)


        CodeStyle Warning in or_in_a_row_warn(): Найдено 1 подряд идущих условий OR. Рекомендуется замена условий на выражение IN (x, y, z). 
        (SQL Style Guide. WHERE/ON. Ссылка: https://wiki.tele2.ru/pages/viewpage.action?pageId=139127857&preview=/139127857/139128786/SQL%20StyleGuide.pdf)
        


WITH all_users_mc AS (
    SELECT t.subs_id,
           t.msisdn,
           t.transaction_id,
           t.transaction_dttm,
           t.amount_rub,
           t.revenue_rub,
           t.update_ts,
           serv.valid_from_date,
           serv.valid_to_date
    FROM prd2_dds_v2.mobile_commerce AS t
    JOIN prd2_dds_v2.service_instance AS serv 
        ON (
        t.subs_id = serv.subs_id
        AND serv.service_id = 30709
        )
    WHERE 1 = 1
        AND t.transaction_dttm > '2022-09-30 00:00:00'
        AND t.dt > 1
),

ordered_movings AS (
    SELECT t.subs_id,
           t.start_dttm,
           t.start_date,
           t.end_dttm,
           t.a_number,
           t.b_number
    FROM prd2_dds_v.usage AS t
    WHERE 1 = 1
        AND t.start_date BETWEEN DATE'2022-10-01' AND DATE'2022-10-31'
        AND t.traffic_type_id = 2
            AND (
    t.b_number = '7757'
    OR t.b_numbe = 11
        )
    AND (
    t.a_number IN (
    '312',
    '7757')
    OR LENGTH(t.a_n

In [63]:
print(sql.raw_bodies[1])

    SELECT t.subs_id,
            t.start_dttm,
            t.start_date,
            t.end_dttm,
            t.a_number,
            t.b_number
     FROM prd2_dds_v.usage AS t
     WHERE t.start_date BETWEEN DATE'2022-10-01' AND DATE'2022-10-31'
         AND t.traffic_type_id = 2
         AND (t.b_number = '7757'
              OR t.b_numbe = 11)
         AND (t.a_number IN ('312',
                             '7757')
              OR length(t.a_number) = 11)


In [64]:
sql.inside_with

[['    SELECT t.subs_id,',
  '            t.msisdn,',
  '            t.transaction_id,',
  '            t.transaction_dttm,',
  '            t.amount_rub,',
  '            t.revenue_rub,',
  '            t.update_ts,',
  '            serv.valid_from_date,',
  '            serv.valid_to_date',
  '     FROM prd2_dds_v2.mobile_commerce AS t',
  '     JOIN prd2_dds_v2.service_instance AS serv ON t.subs_id = serv.subs_id',
  '     AND serv.service_id = 30709',
  "     WHERE t.transaction_dttm > '2022-09-30 00:00:00'",
  '         AND t.dt > 1 '],
 ['    SELECT t.subs_id,',
  '            t.start_dttm,',
  '            t.start_date,',
  '            t.end_dttm,',
  '            t.a_number,',
  '            t.b_number',
  '     FROM prd2_dds_v.usage AS t',
  "     WHERE t.start_date BETWEEN DATE'2022-10-01' AND DATE'2022-10-31'",
  '         AND t.traffic_type_id = 2',
  "         AND (t.b_number = '7757'",
  '              OR t.b_numbe = 11)',
  "         AND (t.a_number IN ('312',",
  "    

In [85]:
print(sql.full_code[1])

AttributeError: 'WithStatementStyle' object has no attribute 'full_code'

In [54]:
sql.inside_with

[['    SELECT t.subs_id,',
  '            t.start_dttm,',
  '            t.start_date,',
  '            t.end_dttm,',
  '            t.a_number,',
  '            t.b_number',
  '     FROM prd2_dds_v.usage AS t',
  '     WHERE 1 = 1',
  "         AND t.start_date BETWEEN DATE'2022-10-01' AND DATE'2022-10-31'",
  '         AND t.traffic_type_id = 2',
  "         AND (t.b_number = '7757'",
  '              OR length(t.a_number) = 11)',
  "         AND (t.a_number IN ('312',",
  "                             '7757')",
  '              OR length(t.a_number) = 11) )',
  'SELECT *',
  'FROM tempo']]

In [52]:
sql.correct_last_block[0]

['    SELECT t.subs_id,',
 '            t.start_dttm,',
 '            t.start_date,',
 '            t.end_dttm,',
 '            t.a_number,',
 '            t.b_number',
 '     FROM prd2_dds_v.usage AS t',
 '     WHERE 1 = 1',
 "         AND t.start_date BETWEEN DATE'2022-10-01' AND DATE'2022-10-31'",
 '         AND t.traffic_type_id = 2',
 "         AND (t.b_number = '7757'",
 '              OR length(t.a_number) = 11)',
 "         AND (t.a_number IN ('312',",
 "                             '7757')",
 '              OR length(t.a_number = 11 ']

In [65]:
sql.correct_last_block[0]

['    SELECT t.foo1,', '            t.buzz1', '     FROM tempo AS t']

In [ ]:
sql.